<a href="https://colab.research.google.com/github/FredBrom/curso_pandas/blob/main/Exemplo_Calcular_o_numero_de_postos_por_Habitante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo - Calcular o numero de postos por 100 mil habitantes 
Data: 10-02-23

Aluno: Frederico Marx Brom Carneiro

Este Exemplo mostra a concatenação de dois bancos de dados, um contendo o numero de postos por cidade e outo contendo o numero de habitantes.


In [1]:
#Importa a biblioteca gera os data-frames

import pandas as pd

comb_df = pd.read_excel('ca-2021-02.xlsx')
habitantes_df = pd.read_csv('ibge_num_habitantes_estimado.csv', delimiter=';')

In [2]:
#Imprime informações sobre os data_frames
print(comb_df.info())
print(habitantes_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168409 entries, 0 to 168408
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Regiao - Sigla     168409 non-null  object        
 1   Estado - Sigla     168409 non-null  object        
 2   Municipio          168409 non-null  object        
 3   Revenda            168409 non-null  object        
 4   CNPJ da Revenda    168409 non-null  object        
 5   Nome da Rua        168409 non-null  object        
 6   Numero Rua         168353 non-null  object        
 7   Complemento        35057 non-null   object        
 8   Bairro             168043 non-null  object        
 9   Cep                168409 non-null  object        
 10  Produto            168409 non-null  object        
 11  Data da Coleta     168409 non-null  datetime64[ns]
 12  Valor de Venda     168409 non-null  float64       
 13  Valor de Compra    0 non-null       float64 

In [3]:
#Renomeia a coluna Estado e NumHabitantes2121 do data-frame habitantes_df
habitantes_df.rename(columns={'Estado':'Estado - Sigla', 'NumHabitantes2021':'Habitantes'}, inplace=True)
print(habitantes_df.keys())


Index(['Municipio', 'Estado - Sigla', 'Habitantes'], dtype='object')


In [4]:
#Remove as colunas desnecessárias do data-frame comb_df
necessary = ['Revenda', 'Municipio', 'Estado - Sigla', 'CNPJ da Revenda']

to_drop = []

for column in comb_df.keys():
  if column not in necessary:
    to_drop.append(column)

comb_df.drop(to_drop, axis='columns', inplace=True)

print('Colunas Removidas: ',to_drop)

print(comb_df.info())

Colunas Removidas:  ['Regiao - Sigla', 'Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro', 'Cep', 'Produto', 'Data da Coleta', 'Valor de Venda', 'Valor de Compra', 'Unidade de Medida', 'Bandeira']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168409 entries, 0 to 168408
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Estado - Sigla   168409 non-null  object
 1   Municipio        168409 non-null  object
 2   Revenda          168409 non-null  object
 3   CNPJ da Revenda  168409 non-null  object
dtypes: object(4)
memory usage: 5.1+ MB
None


In [5]:
#Combina os dois data-frames, mantendo apenas as entradas que aparecem nos dois bancos de dados
# usar MERGE e INNER

merged_df = comb_df.merge(habitantes_df, how='inner')
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20634 entries, 0 to 20633
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Estado - Sigla   20634 non-null  object
 1   Municipio        20634 non-null  object
 2   Revenda          20634 non-null  object
 3   CNPJ da Revenda  20634 non-null  object
 4   Habitantes       20634 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 967.2+ KB
None


In [6]:
#Remove postos duplicados, Utilizando CNPJ da Revenda, pois o CNPJ é unico para um mesmo posto.

merged_df.drop_duplicates(subset='CNPJ da Revenda', inplace=True)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 0 to 20608
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Estado - Sigla   1532 non-null   object
 1   Municipio        1532 non-null   object
 2   Revenda          1532 non-null   object
 3   CNPJ da Revenda  1532 non-null   object
 4   Habitantes       1532 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 71.8+ KB
None


In [7]:
numero_postos = merged_df.groupby(by=['Estado - Sigla', 'Municipio', 'Habitantes'])['CNPJ da Revenda'].count()

Estado = []
Municipio = []
Habitantes = []
Num_Postos = []

for estado_UF, cidade, populacao in numero_postos.keys():
  Estado.append(estado_UF)
  Municipio.append(cidade)
  Habitantes.append(populacao)
  Num_Postos.append(numero_postos[estado_UF,cidade,populacao])

postos_por_habitante = []

for i in range(len(Municipio)):
  postos_por_habitante.append((Num_Postos[i]*100e3)/Habitantes[i])

postos_df = pd.DataFrame()

postos_df['Estado - Sigla'] = Estado
postos_df['Municipio'] = Municipio
postos_df['Habitantes'] = Habitantes
postos_df['Numero de Postos'] = Num_Postos
postos_df['Postos por 100 mil Habitantes'] = postos_por_habitante


In [8]:
print(postos_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Estado - Sigla                 15 non-null     object 
 1   Municipio                      15 non-null     object 
 2   Habitantes                     15 non-null     int64  
 3   Numero de Postos               15 non-null     int64  
 4   Postos por 100 mil Habitantes  15 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 728.0+ bytes
None


##Apresentação dos Dados do Data-Frame

In [9]:
display(postos_df)

,Estado - Sigla,Municipio,Habitantes,Numero de Postos,Postos por 100 mil Habitantes
0,AC,RIO BRANCO,419452,44,10.489877
1,GO,GOIANIA,1555626,93,5.978301
2,MG,BELO HORIZONTE,2530701,54,2.133796
3,MS,CAMPO GRANDE,916001,104,11.353699
4,PR,CURITIBA,1963726,110,5.601596
5,RJ,RIO DE JANEIRO,6775561,379,5.593633
6,RN,NATAL,803739,37,4.603484
7,RO,PORTO VELHO,548952,40,7.286612
8,RS,PORTO ALEGRE,1492530,73,4.891024
9,SP,CAMPINAS,1223237,51,4.169266
